In [68]:
import pandas as pd
from utils import connection as con
from etl import extract as ex
from etl import load
from etl import transform as t

In [70]:
conn_db = con.connect_db()
conn_dw = con.connect_dw()

In [71]:
# Pasamos los datos de las tablas a su DataFrame

df_categoria = ex.extract_only_table("Categoria", conn_db)
df_subcategoria = ex.extract_only_table("SubCategoria", conn_db)
df_carta = ex.extract_only_table("Carta", conn_db)

df_mesa = ex.extract_only_table("Mesa", conn_db)
df_clientes = ex.extract_only_table("Clientes", conn_db)
df_distrito = ex.extract_only_table("Distrito", conn_db).rename(columns={'Descripcion' : 'Distrito'})
df_direccion = ex.extract_only_table("Direcciones", conn_db)
df_local = ex.extract_only_table("Local", conn_db)
df_trabajador = ex.extract_only_table("Trabajador", conn_db)

df_pedidosdetalle = ex.extract_only_table("PedidosDetalle", conn_db)


Reemplazando IDs por descripciones

In [72]:
# Reemplazamos el idDistrito por su descripción.
df_direccion_dw = df_direccion.merge(df_distrito, on='idDistrito', how='left')[['idDirecciones','Descripcion','Distrito','idClientes','Estado']]

df_local_dw = df_local.merge(df_distrito, on='idDistrito', how='left')[['idLocal','NombreLocal','Direccion','Distrito','HorarioAtencion']]


In [73]:
# Reemplazamos el idCategoria y idSubCategoria para la carta y ponemos sus descripciones.
df_carta_dw = df_carta.merge(df_subcategoria.rename(columns={'nombre' : 'SubCategoria'}), on='idSubCategoria', how='left')\
                      .merge(df_categoria.rename(columns={'Nombre' : 'Categoria'}), on='idCategoria', how='left')\
                      [['idCarta','Nombre','Descripcion','Precio','Estado','Categoria','SubCategoria']]


In [74]:
# Reemplazamos el idLocal por su descripción.
df_trabajador_dw = df_trabajador.merge(df_local.rename(columns={'NombreLocal' : 'Local'}), on='idLocal', how='left')\
                                [['idTrabajador','ApellidosNombres','Local']]


Cargando los datos a las dimensiones

In [95]:
load.load_dw_restaurante("Dim_Carta", conn_dw, df_carta_dw)

AttributeError: module 'etl.load' has no attribute 'load_dw_restaurante'

In [77]:
load.load_dw_restaurante("Dim_Mesa", conn_dw, df_mesa)

AttributeError: module 'etl.load' has no attribute 'load_dw_restaurante'

In [ ]:
load.load_dw_restaurante("Dim_Local", conn_dw, df_local_dw)

In [ ]:
load.load_dw_restaurante("Dim_Trabajador", conn_dw, df_trabajador_dw)

In [ ]:
load.load_dw_restaurante("Dim_Clientes", conn_dw, df_clientes)

In [ ]:
load.load_dw_restaurante("Dim_Direcciones", conn_dw, df_direccion_dw)

In [ ]:
load.load_dw_restaurante("Dim_PedidosDetalle", conn_dw, df_pedidosdetalle)

In [8]:
fruits = ["apple", "banana", "cherry"]
for x in fruits:
  print(x)

apple
banana
cherry


In [ ]:
df_join  = orders_df.merge(order_items_df, left_on='order_id', right_on='order_item_order_id')\
                    .merge(products_df, left_on='order_item_product_id', right_on='product_id')\
                    .merge(categories_df, left_on='product_category_id', right_on='category_id')\
                    .merge(departments_df, left_on='category_department_id', right_on='department_id')
df_join